In [ ]:
using DrWatson
using Plots
using DataFrames
import PartitionedArraysBenchmarks as pb

In [ ]:
path = mkpath("results")
cmd = :bash
function reduce_timing(ts)
    nworkers = length(ts)
    nruns = length(first(ts))
    t = minimum(ir->maximum(iw->ts[iw][ir],1:nworkers),1:nruns)
    t
end
filename = params -> joinpath(path,pb.jobname(params))
function parts_per_dir_array(P,D)
    cis = CartesianIndices(P)
    a = filter(cis) do ci
        t = Tuple(ci)
        issorted(t) && maximum(t)-minimum(t)<=1
    end
    map(a) do a
        ntuple(D) do d
            if d > length(P)
                return 1
            end
            Int(a[d])
        end
    end
end

## SpMV

In [ ]:
parts_per_dir_space = [(1,1,1),(2,1,1),(3,1,1),(4,1,1),(5,1,1)]
parts_per_dir_space = parts_per_dir_array((2,2,2),3)
methods_space = ["PartitionedArrays","Petsc"]
param_space = Dict(
    :nodes => 1,
    :ntasks_per_node => Derived(:parts_per_dir,prod),
    :np => Derived(:parts_per_dir,prod),
    :method => methods_space,
    :cells_per_dir => [(100,100,100)],
    :parts_per_dir => parts_per_dir_space,
    :nruns => 10,
    :benchmark => :benchmark_spmv,
)
params = dict_list(param_space)
for params in params
    produce_or_load(params;filename) do params
        pb.runjob(cmd,params;filename)
        Dict{String,Any}()
    end
end

In [ ]:
files = readdir(path)
files = filter(f->occursin("_results",f),files)
dicts = map(f->load(joinpath(path,f)),files)
dicts = filter(d->d["benchmark"]=== :benchmark_spmv,dicts)
df = DataFrame(dicts)
df[!,:spmv] = reduce_timing.(df[!,:spmv])
sort!(df,[:method,:np])
df

In [ ]:
cells_per_dir = (100,100,100)
plt = plot(size=(400,300),xlabel="P",ylabel="Wall time [s]")
for method in param_space[:method]
    take_row(row) = row.method == method && row.cells_per_dir == cells_per_dir
    df_filtered = filter(take_row,df)
    sort!(df_filtered,[:np])
    x = df_filtered[:,:np]
    y = df_filtered[:,:spmv]
    plot!(x,y,label=method,markers=:auto)
end
plt

## Matrix assembly

In [ ]:
parts_per_dir_space = [(1,1,1),(2,1,1),(3,1,1),(4,1,1)]
parts_per_dir_space = parts_per_dir_array((2,2,2),3)
methods_space = ["psprse","petsc_coo"]
param_space = Dict(
    :nodes => 1,
    :ntasks_per_node => Derived(:parts_per_dir,prod),
    :np => Derived(:parts_per_dir,prod),
    :method => methods_space,
    :cells_per_dir => [(100,100,100)],
    :parts_per_dir => parts_per_dir_space,
    :nruns => 10,
    :benchmark => :benchmark_psparse,
)
params = dict_list(param_space)
for params in params
    produce_or_load(params;filename) do params
        pb.runjob(:cmd,params;filename)
        Dict{String,Any}()
    end
end

In [ ]:
files = readdir(path)
files = filter(f->occursin("_results",f),files)
dicts = map(f->load(joinpath(path,f)),files)
dicts = filter(d->d["benchmark"]=== :benchmark_psparse,dicts)
df = DataFrame(dicts)
df[!,:buildmat] = reduce_timing.(df[!,:buildmat])
df[!,:rebuildmat] = reduce_timing.(df[!,:rebuildmat])
sort!(df,[:method,:np])

In [ ]:
cells_per_dir = (100,100,100)
plt = plot(size=(400,300),xlabel="P",ylabel="Wall time [s]")
for method in param_space[:method]
    take_row(row) = row.method == method && row.cells_per_dir == cells_per_dir
    df_filtered = filter(take_row,df)
    sort!(df_filtered,[:np])
    x = df_filtered[:,:np]
    y = df_filtered[:,:buildmat]
    plot!(x,y,label=method,markers=:auto)
end
plt